# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 10:54AM,256280,10,SO94499,"Senseonics, Incorporated",Released
1,Jan 31 2023 10:53AM,256279,20,9098845,"Exact-Rx, Inc.",Released
2,Jan 31 2023 10:53AM,256279,20,9098872,"Exact-Rx, Inc.",Released
3,Jan 31 2023 10:52AM,256278,16,9081365,Sartorius Bioprocess Solutions,Released
4,Jan 31 2023 10:52AM,256278,16,9086209,Sartorius Bioprocess Solutions,Released
5,Jan 31 2023 10:43AM,256276,18,ECI-9098318,ECI Pharmaceuticals LLC,Released
6,Jan 31 2023 10:30AM,256273,10,PRF-50664,Bio-PRF,Released
7,Jan 31 2023 10:30AM,256273,10,PRF-50684,Bio-PRF,Released
8,Jan 31 2023 10:30AM,256273,10,PRF-50703,Bio-PRF,Released
9,Jan 31 2023 10:30AM,256273,10,PRF-50709,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,256274,Released,1
32,256276,Released,1
33,256278,Released,2
34,256279,Released,2
35,256280,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256274,NaN,1.0
256276,NaN,1.0
256278,NaN,2.0
256279,NaN,2.0
256280,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256149,0.0,2.0
256198,0.0,2.0
256218,0.0,1.0
256236,5.0,24.0
256240,12.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256149,0,2
256198,0,2
256218,0,1
256236,5,24
256240,12,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256149,0,2
1,256198,0,2
2,256218,0,1
3,256236,5,24
4,256240,12,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256149,,2
1,256198,,2
2,256218,,1
3,256236,5,24
4,256240,12,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated"
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc."
3,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions
5,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC
6,Jan 31 2023 10:30AM,256273,10,Bio-PRF
20,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.
21,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC"
30,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc."
31,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation
33,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",,1
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc.",,2
2,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,,2
3,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,,1
4,Jan 31 2023 10:30AM,256273,10,Bio-PRF,,14
5,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,,1
6,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",3,6
7,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",,1
8,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,,2
9,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",1,
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc.",2,
2,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,2,
3,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,1,
4,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14,
5,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,1,
6,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",6,3
7,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",1,
8,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,2,
9,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation,16,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",1,
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc.",2,
2,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,2,
3,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,1,
4,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",1.0,NaN
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc.",2.0,NaN
2,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,2.0,NaN
3,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,1.0,NaN
4,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",1.0,0.0
1,Jan 31 2023 10:53AM,256279,20,"Exact-Rx, Inc.",2.0,0.0
2,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,2.0,0.0
3,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,1.0,0.0
4,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4100032,125.0,8.0
12,256256,2.0,8.0
15,768776,118.0,22.0
16,256278,2.0,0.0
18,256276,1.0,0.0
19,1281298,33.0,5.0
20,256279,2.0,0.0
21,256218,1.0,0.0
22,256245,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4100032,125.0,8.0
1,12,256256,2.0,8.0
2,15,768776,118.0,22.0
3,16,256278,2.0,0.0
4,18,256276,1.0,0.0
5,19,1281298,33.0,5.0
6,20,256279,2.0,0.0
7,21,256218,1.0,0.0
8,22,256245,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,125.0,8.0
1,12,2.0,8.0
2,15,118.0,22.0
3,16,2.0,0.0
4,18,1.0,0.0
5,19,33.0,5.0
6,20,2.0,0.0
7,21,1.0,0.0
8,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,125.0
1,12,Released,2.0
2,15,Released,118.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,8.0,8.0,22.0,0.0,0.0,5.0,0.0,0.0,0.0
Released,125.0,2.0,118.0,2.0,1.0,33.0,2.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,8.0,8.0,22.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Released,125.0,2.0,118.0,2.0,1.0,33.0,2.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,8.0,8.0,22.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Released,125.0,2.0,118.0,2.0,1.0,33.0,2.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()